# Соревнование по классификации новостей

## Импорты, константы

In [44]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [45]:
RANDOM_STATE = 42

## Подготовка данных

Считаем три датасета с новостями из lenta.ru

In [3]:
lenta1 = pd.read_csv('lenta_news.csv')
lenta2 = pd.read_csv('lenta_news_1.csv')
lenta3 = pd.read_csv('lenta_news_2.csv')

In [22]:
lenta1.columns = ['topic', 'url', 'text']
lenta2.columns = ['topic', 'url', 'text']
lenta3.columns = ['topic', 'url', 'text']

In [25]:
lenta = pd.concat([lenta1, lenta2, lenta3])

In [26]:
lenta

,topic,url,text
0,Мир,https://lenta.ru/news/2024/01/02/early/,"Президент Венесуэлы Николас Мадуро заявил, что..."
1,Бывший СССР,https://lenta.ru/news/2024/01/02/zelen_/,Президент Украины Владимир Зеленский в интервь...
2,Интернет и СМИ,https://lenta.ru/news/2024/01/02/zapretnoe-slo...,Президент Украины Владимир Зеленский пришел в ...
3,Бывший СССР,https://lenta.ru/news/2024/01/02/instead/,"Президент Украины Владимир Зеленский заявил, ч..."
4,Бывший СССР,https://lenta.ru/news/2024/01/02/zelenn/,"Президент Украины Владимир Зеленский заявил, ч..."
...,...,...,...
97536,Ценности,https://lenta.ru/news/2021/10/13/reshetova/,"Бывшая девушка рэпера Тимати, фотомодель Анаст..."
97537,Экономика,https://lenta.ru/news/2021/10/13/ecpere/,Страны G7 потребовали обеспечить способность э...
97538,Путешествия,https://lenta.ru/news/2021/10/13/fedorvpered/,Российский путешественник Федор Конюхов назвал...
97539,Экономика,https://lenta.ru/news/2021/10/13/index/,"Курс евро вырос до 83,41 рубля (плюс 0,53 рубл..."


In [27]:
lenta = lenta.drop_duplicates()
lenta = lenta.dropna()

In [28]:
lenta.shape

(291652, 3)

In [29]:
lenta['topic'].unique()

array(['Мир', 'Бывший СССР', 'Интернет и СМИ', 'Культура', 'Россия',
       'Экономика', 'Наука и техника', 'Среда обитания', 'Спорт',
       'Путешествия', 'Забота о себе', 'Силовые структуры', 'Ценности',
       'Из жизни', 'Моя страна', '69-я параллель', 'Бизнес', 'Оружие',
       'Нацпроекты', 'Мотор'], dtype=object)

Избавимся от ненужных колонок в датасете

In [30]:
lenta = lenta[~lenta['topic'].isin(['Мир', 'Интернет и СМИ', 'Культура', 'Среда обитания', 'Ценности', 
                                    'Из жизни', 'Моя страна', '69-я параллель', 'Бизнес', 'Оружие', 'Нацпроекты', 'Мотор'])]

In [33]:
lenta = lenta.drop('url', axis=1)

In [35]:
lenta = lenta.reset_index(drop=True)

Считаем датасет с новостями из fontanka.ru

In [37]:
fontanka = pd.read_csv('fontanka_news.csv')

In [39]:
fontanka = fontanka.drop(['date', 'title', 'url', 'time', 'comm_num', 'author', 'views'], axis=1)

In [41]:
fontanka = fontanka.drop_duplicates()
fontanka = fontanka.dropna()
fontanka = fontanka.rename(columns={'content': 'text'})

In [43]:
fontanka.shape

(117991, 2)

In [44]:
fontanka['topic'].unique()

array(['Происшествия', 'Авто', 'Общество', 'Доктор Питер', 'Власть',
       'Строительство', 'Город', 'Доброе дело', 'Бизнес', 'Афиша Plus',
       'topic', 'Образ жизни', 'Работа', 'Спорт', 'Недвижимость',
       'Финансы', 'Туризм', 'Политика', 'Технологии', 'ЖКХ',
       'Особое мнение', 'Новости компаний', 'Бизнес-трибуна', 'Финляндия',
       'Открытое письмо'], dtype=object)

In [45]:
fontanka = fontanka[~fontanka['topic'].isin(['Происшествия', 'Авто', 'Доктор Питер', 'Власть', 'Город', 'Доброе дело', 
                                             'Бизнес', 'Афиша Plus', 'topic', 'Образ жизни', 'Работа', 'Недвижимость', 
                                             'Финансы', 'Политика', 'ЖКХ', 'Особое мнение', 'Новости компаний', 
                                             'Бизнес-трибуна', 'Финляндия', 'Открытое письмо'])]

In [49]:
fontanka = fontanka.reset_index(drop=True)
fontanka

,topic,text
0,Общество,Зампред Совбеза РФ Дмитрий Медведев высказался...
1,Общество,Фото: Анжела Мнацаканян/ Фонтанка.руПоделиться...
2,Строительство,"19 октября в Setl Group состоялся День донора,..."
3,Общество,Жительница города Бирск в Башкирии нашла в лес...
4,Общество,В Петербурге женщину похоронили в одной могиле...
...,...,...
37697,Спорт,\tФутбольный клуб «Зенит» 12 декабря прокоммен...
37698,Общество,\tРоскомнадзор рассматривает вариант блокировк...
37699,Общество,\tЧиновника Ространснадзора уволили в связи с ...
37700,Спорт,\tНападающий «Вашингтона» Александр Овечкин в ...


Считаем датасет с новостями про строительство из asninfo.ru

In [50]:
asn = pd.read_csv('builds.csv')

In [52]:
asn.columns = ['topic', 'url', 'text']

In [54]:
asn = asn.drop_duplicates()
asn = asn.dropna()
asn = asn.drop(['url'], axis=1)
asn = asn.reset_index(drop=True)

In [55]:
asn

,topic,text
0,Строительство,\n\nНа территории жилого массива в границах Сч...
1,Строительство,\n\nЗавершение строительства ЖК «Оранж» продол...
2,Строительство,\n\n\n\nблагоустройство\n\nНа строящейся Южной...
3,Строительство,\n\nВ ходе производственной практики магистран...
4,Строительство,\n\n\n\nблагоустройство\n\nДетский сад с бассе...
...,...,...
8511,Строительство,\n\nВ рамках рабочей поездки в Великие Луки гу...
8512,Строительство,\n\nНа заседании регионального правительства м...
8513,Строительство,\n\nНа территории Новой Москвы в поселке Маруш...
8514,Строительство,\n\nВ приусадебном парке Демидовых в Тайцах Га...


In [56]:
news = pd.concat([lenta, fontanka, asn])
news = news.reset_index(drop=True)

In [58]:
news['topic'].unique()

array(['Бывший СССР', 'Россия', 'Экономика', 'Наука и техника', 'Спорт',
       'Путешествия', 'Забота о себе', 'Силовые структуры', 'Общество',
       'Строительство', 'Туризм', 'Технологии'], dtype=object)

In [59]:
news['topic'] = news['topic'].replace({'Бывший СССР': '3', 
                                       'Россия':'0', 
                                       'Экономика':'1', 
                                       'Наука и техника':'8', 
                                       'Спорт':'4',
                                       'Путешествия':'7', 
                                       'Забота о себе':'5', 
                                       'Силовые структуры':'2', 
                                       'Общество':'0',
                                       'Строительство':'6', 
                                       'Туризм':'7', 
                                       'Технологии':'8'
                                       })

In [60]:
news

,topic,text
0,3,Президент Украины Владимир Зеленский в интервь...
1,3,"Президент Украины Владимир Зеленский заявил, ч..."
2,3,"Президент Украины Владимир Зеленский заявил, ч..."
3,3,Полковник Службы безопасности Украины (СБУ) в ...
4,3,Украинский президент Владимир Зеленский попрос...
...,...,...
230836,6,\n\nВ рамках рабочей поездки в Великие Луки гу...
230837,6,\n\nНа заседании регионального правительства м...
230838,6,\n\nНа территории Новой Москвы в поселке Маруш...
230839,6,\n\nВ приусадебном парке Демидовых в Тайцах Га...


Перемешаем все новости внутри датасета

In [61]:
news = news.sample(frac=1).reset_index(drop=True)
news


,topic,text
0,0,Транспортировку газа по трубопроводу «Сила Сиб...
1,2,Глава Следственного комитета России (СКР) Алек...
2,0,В Красноярске в ковидном госпитале произошел п...
3,4,\tОткрытие центра выдачи паспортов болельщиков...
4,0,Российским военным удалось уничтожить десант В...
...,...,...
230836,5,Кандидат медицинских наук и спортивный врач Ал...
230837,0,Телеведущий Иван Ургант опубликовал в Instagra...
230838,0,"Часть подразделений Вооруженных сил России, в ..."
230839,6,\n\nГород выставит на торги по аренде земельны...


Сохраним датасет

In [62]:
news.to_csv('news_unbalanced.csv', index=False)

## Классы

Разметка классов была приведена к следующей:

- 'Общество/Россия' : 0
- 'Экономика' : 1
- 'Силовые структуры' : 2
- 'Бывший СССР' : 3
- 'Спорт' : 4
- 'Забота о себе' : 5
- 'Строительство' : 6
- 'Туризм/Путешествия' : 7
- 'Наука и техника' : 8

In [63]:
df_u = pd.read_csv('news_unbalanced.csv')

In [64]:
df_u['topic'].value_counts()

0    86662
3    33008
1    29364
4    19641
2    18046
8    15186
7    11291
6    11253
5     6390
Name: topic, dtype: int64

Создадим еще один датасет, но уже со сбалансированными классами.
Балансировку проведем при помощи удаления данных.

In [65]:
df = df_u.copy()

rows_to_remove = df.loc[df['topic'] == 0].sample(n=80272)
df = df.drop(rows_to_remove.index)

rows_to_remove = df.loc[df['topic'] == 3].sample(n=26618)
df = df.drop(rows_to_remove.index)

rows_to_remove = df.loc[df['topic'] == 1].sample(n=22974)
df = df.drop(rows_to_remove.index)

rows_to_remove = df.loc[df['topic'] == 4].sample(n=13251)
df = df.drop(rows_to_remove.index)

rows_to_remove = df.loc[df['topic'] == 2].sample(n=11656)
df = df.drop(rows_to_remove.index)

rows_to_remove = df.loc[df['topic'] == 8].sample(n=8796)
df = df.drop(rows_to_remove.index)

rows_to_remove = df.loc[df['topic'] == 7].sample(n=4901)
df = df.drop(rows_to_remove.index)

rows_to_remove = df.loc[df['topic'] == 6].sample(n=4863)
df = df.drop(rows_to_remove.index)

In [66]:
df['topic'].value_counts()

4    6390
3    6390
7    6390
2    6390
1    6390
0    6390
6    6390
8    6390
5    6390
Name: topic, dtype: int64

In [67]:
df.to_csv('news_balanced.csv', index=False)

## Baseline решение (BoW + LogReg)

### Для сбалансированных данных

In [3]:
X = df[['text']]
y = df['topic']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

NameError: name 'df' is not defined

In [71]:
vec = CountVectorizer()
vec.fit(X_train['text'])

bow = vec.transform(X_train['text'])  # bow — bag of words (мешок слов)
bow_test = vec.transform(X_test['text'])

print(bow.shape)

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

clf = LogisticRegression(max_iter=200, random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(bow_test)

print(classification_report(y_test, pred))

(43132, 289278)
              precision    recall  f1-score   support

           0       0.77      0.73      0.75      1647
           1       0.90      0.92      0.91      1581
           2       0.91      0.95      0.93      1591
           3       0.88      0.92      0.90      1611
           4       0.98      0.97      0.98      1587
           5       0.97      0.98      0.97      1572
           6       0.97      0.98      0.97      1572
           7       0.96      0.94      0.95      1645
           8       0.95      0.92      0.93      1572

    accuracy                           0.92     14378
   macro avg       0.92      0.92      0.92     14378
weighted avg       0.92      0.92      0.92     14378



In [72]:
Test = pd.read_csv("fromkaggle/test_news.csv")

In [75]:
vec = CountVectorizer()
vec.fit(X['text'])

bow = vec.transform(X['text'])  # bow — bag of words (мешок слов)
bow_test = vec.transform(Test['content'])

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

clf = LogisticRegression(max_iter=200, random_state=42)
clf.fit(bow, y)
pred = clf.predict(bow_test)

In [77]:
subm = pd.read_csv("fromkaggle/base_submission_news.csv")

In [79]:
subm['topic'] = pred

subm.to_csv("submissions/my_baseline_submission_0.csv", index=False)

### Для несбалансированных данных

In [46]:
X = df_u[['text']]
y = df_u['topic']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [83]:
vec = CountVectorizer()
vec.fit(X_train['text'])

bow = vec.transform(X_train['text'])  # bow — bag of words (мешок слов)
bow_test = vec.transform(X_test['text'])

print(bow.shape)

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

clf = LogisticRegression(max_iter=500, random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(bow_test)

print(classification_report(y_test, pred))

(173130, 525287)
              precision    recall  f1-score   support

           0       0.87      0.90      0.89     21623
           1       0.91      0.90      0.90      7358
           2       0.91      0.88      0.90      4541
           3       0.88      0.89      0.88      8198
           4       0.98      0.97      0.97      4885
           5       0.94      0.92      0.93      1585
           6       0.96      0.93      0.94      2842
           7       0.94      0.88      0.91      2858
           8       0.93      0.87      0.90      3821

    accuracy                           0.90     57711
   macro avg       0.92      0.90      0.91     57711
weighted avg       0.90      0.90      0.90     57711



In [86]:
Test = pd.read_csv("fromkaggle/test_news.csv")

In [87]:
vec = CountVectorizer()
vec.fit(X['text'])

bow = vec.transform(X['text'])  # bow — bag of words (мешок слов)
bow_test = vec.transform(Test['content'])

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

clf = LogisticRegression(max_iter=500, random_state=42)
clf.fit(bow, y)
pred = clf.predict(bow_test)

In [88]:
subm['topic'] = pred

subm.to_csv("submissions/my_baseline_submission_1.csv", index=False)

In [89]:
df_u.shape

(230841, 2)

## Решение 2. (Tfidf + LogReg)

In [92]:
vec = TfidfVectorizer(ngram_range=(1, 1))
vec_train = vec.fit_transform(X_train['text'])
vec_test = vec.transform(X_test['text'])

scaler = MaxAbsScaler()
vec_train = scaler.fit_transform(vec_train)
vec_test = scaler.transform(vec_test)

clf = LogisticRegression(max_iter=500, random_state=42)
clf.fit(vec_train, y_train)
pred_tfidf = clf.predict(vec_test)
print(classification_report(y_test, pred_tfidf))

              precision    recall  f1-score   support

           0       0.89      0.91      0.90     21623
           1       0.91      0.92      0.92      7358
           2       0.93      0.90      0.91      4541
           3       0.89      0.89      0.89      8198
           4       0.98      0.97      0.98      4885
           5       0.94      0.94      0.94      1585
           6       0.96      0.96      0.96      2842
           7       0.94      0.89      0.92      2858
           8       0.92      0.88      0.90      3821

    accuracy                           0.91     57711
   macro avg       0.93      0.92      0.92     57711
weighted avg       0.91      0.91      0.91     57711



In [93]:
Test = pd.read_csv("fromkaggle/test_news.csv")

In [95]:
vec = TfidfVectorizer(ngram_range=(1, 1))
vec.fit(X['text'])

vec_train = vec.fit_transform(X['text'])
vec_test = vec.transform(Test['content'])

scaler = MaxAbsScaler()
vec_train = scaler.fit_transform(vec_train)
vec_test = scaler.transform(vec_test)

clf = LogisticRegression(max_iter=500, random_state=42)
clf.fit(vec_train, y)
pred = clf.predict(vec_test)

In [ ]:
subm = pd.read_csv("fromkaggle/base_submission_news.csv")

In [ ]:
subm['topic'] = pred

subm.to_csv("submissions/my_baseline_submission_2.csv", index=False)